In [ ]:
#filepath="Best_weight-{epoch:02d}-{val_auroc:.2f}.hdf5"
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 30
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
X_train.shape,X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [ ]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [ ]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [ ]:
model = Model(inputs=input, outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 12)   324         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 12)  48          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 12)   0           ['batch_normalization[0][0]']

In [ ]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=100,
                    verbose=1, 
                    validation_data=(X_test, y_test))

Epoch 1/100
391/391 [==============================] - 129s 229ms/step - loss: 1.7648 - accuracy: 0.3471 - val_loss: 2.0367 - val_accuracy: 0.3052
Epoch 2/100
391/391 [==============================] - 86s 220ms/step - loss: 1.4214 - accuracy: 0.4739 - val_loss: 1.5315 - val_accuracy: 0.4604
Epoch 3/100
391/391 [==============================] - 86s 220ms/step - loss: 1.2394 - accuracy: 0.5472 - val_loss: 1.5906 - val_accuracy: 0.4441
Epoch 4/100
391/391 [==============================] - 86s 219ms/step - loss: 1.1207 - accuracy: 0.5971 - val_loss: 1.4406 - val_accuracy: 0.5141
Epoch 5/100
391/391 [==============================] - 86s 220ms/step - loss: 1.0459 - accuracy: 0.6244 - val_loss: 1.2323 - val_accuracy: 0.5900
Epoch 6/100
391/391 [==============================] - 86s 219ms/step - loss: 0.9907 - accuracy: 0.6428 - val_loss: 1.0826 - val_accuracy: 0.6264
Epoch 7/100
391/391 [==============================] - 86s 220ms/step - loss: 0.9554 - accuracy: 0.6562 - val_loss: 1.0039 

In [ ]:
model.save("model_train.h5")

In [ ]:
model = tf.keras.models.load_model("/content/model_train.h5")

In [ ]:
#start with last epochs 
model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=100,
                    verbose=1, 
                    validation_data=(X_test, y_test))

Epoch 1/100
391/391 [==============================] - 93s 225ms/step - loss: 0.4221 - accuracy: 0.8498 - val_loss: 0.6362 - val_accuracy: 0.8100
Epoch 2/100
391/391 [==============================] - 87s 221ms/step - loss: 0.4165 - accuracy: 0.8526 - val_loss: 0.6657 - val_accuracy: 0.8079
Epoch 3/100
391/391 [==============================] - 87s 221ms/step - loss: 0.4163 - accuracy: 0.8517 - val_loss: 0.6580 - val_accuracy: 0.8076
Epoch 4/100
391/391 [==============================] - 87s 221ms/step - loss: 0.4158 - accuracy: 0.8526 - val_loss: 0.6126 - val_accuracy: 0.8142
Epoch 5/100
391/391 [==============================] - 87s 221ms/step - loss: 0.4143 - accuracy: 0.8532 - val_loss: 0.7632 - val_accuracy: 0.7747
Epoch 6/100
391/391 [==============================] - 87s 222ms/step - loss: 0.4142 - accuracy: 0.8537 - val_loss: 0.6562 - val_accuracy: 0.8030
Epoch 7/100
391/391 [==============================] - 86s 221ms/step - loss: 0.4122 - accuracy: 0.8537 - val_loss: 0.6642 -

In [ ]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (5,5), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output


In [ ]:
num_filter = 12
dropout_rate = 0
l = 12
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (5,5), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, 32, dropout_rate)
First_Transition = transition(First_Block, 32, dropout_rate)

Second_Block = denseblock(First_Transition, 16, dropout_rate)
Second_Transition = transition(Second_Block, 16, dropout_rate)

Third_Block = denseblock(Second_Transition, 16, dropout_rate)
Third_Transition = transition(Third_Block, 8, dropout_rate)

Last_Block = denseblock(Third_Transition,  8, dropout_rate)
output = output_layer(Last_Block)

In [ ]:
tf.keras.backend.clear_session()
model_2 = Model(inputs = input,outputs = output)
model_2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 12)   900         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 12)  48          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 12)   0           ['batch_normalization[0][0]']

In [ ]:
def preprocess_image(x_train,x_test):


  """

    x_train --> training dataframe - type pandas -core-DataFrame

    x_test --> test dataframe - type pandas -core-DataFrame

  
  """

  #preprocess the image scaling
  #Reference : https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
  train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                featurewise_center=False,  # set input mean to 0 over the dataset
                samplewise_center=False,  # set each sample mean to 0
                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                samplewise_std_normalization=False,  # divide each input by its std
                zca_whitening=False,  # apply ZCA whitening
                rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
                zoom_range = 0.2, # Randomly zoom image 
                width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
                height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
                horizontal_flip=True,  # randomly flip images
                vertical_flip=False) 
  

  train_generator.fit(x_train)

 
  validation_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

  validation_generator.fit(x_test)

  return train_generator,validation_generator

In [ ]:
train_generator,validation_generator = preprocess_image(X_train,X_test)

In [ ]:
import datetime
batch_size = 512
filepath="Best_weight-{epoch:02d}-{val_accuracy:.2f}.hdf5"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')



log_dir = "CnnCipher10" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
                            log_dir=log_dir, histogram_freq=1)


model_2.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

model_2.fit(train_generator.flow(X_train,y_train,batch_size= batch_size),
            epochs= 100,verbose= 1,validation_data=validation_generator.flow(X_test,y_test,batch_size = 256),
            callbacks = [cp_callback,tensorboard_callback])

Epoch 1/100
98/98 [==============================] - ETA: 0s - loss: 1.8671 - accuracy: 0.3007
Epoch 00001: val_accuracy improved from -inf to 0.10450, saving model to Best_weight-01-0.10.hdf5
98/98 [==============================] - 291s 2s/step - loss: 1.8671 - accuracy: 0.3007 - val_loss: 3.2265 - val_accuracy: 0.1045
Epoch 2/100
98/98 [==============================] - ETA: 0s - loss: 1.5904 - accuracy: 0.4140
Epoch 00002: val_accuracy improved from 0.10450 to 0.17500, saving model to Best_weight-02-0.17.hdf5
98/98 [==============================] - 216s 2s/step - loss: 1.5904 - accuracy: 0.4140 - val_loss: 2.6548 - val_accuracy: 0.1750
Epoch 3/100
98/98 [==============================] - ETA: 0s - loss: 1.4609 - accuracy: 0.4692
Epoch 00003: val_accuracy improved from 0.17500 to 0.31360, saving model to Best_weight-03-0.31.hdf5
98/98 [==============================] - 216s 2s/step - loss: 1.4609 - accuracy: 0.4692 - val_loss: 1.9886 - val_accuracy: 0.3136
Epoch 4/100
98/98 [======

In [ ]:
model_2.save("model_2_100.h5")
#https://github.com/aayushs879/Densenet-on-CIFAR-10/blob/master/DNST.ipynb

In [ ]:
model_2 = tf.keras.models.load_model("/content/model_2_50.h5")

In [ ]:
import os
checkpoint_path = "training_2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor = 'val_loss',
                                                 save_weights_only=True,
                                                 verbose=1)
#cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')



In [ ]:
model_2.fit(train_generator.flow(X_train,y_train,batch_size= batch_size),
            epochs= 50,verbose= 1,validation_data=validation_generator.flow(X_test,y_test,batch_size = 256),
            callbacks = [cp_callback,tensorboard_callback])

Epoch 1/50
196/196 [==============================] - 328s 2s/step - loss: 0.4901 - accuracy: 0.8307 - val_loss: 0.5754 - val_accuracy: 0.8110

Epoch 00001: saving model to training_2/cp.ckpt
Epoch 2/50
196/196 [==============================] - 325s 2s/step - loss: 0.4850 - accuracy: 0.8314 - val_loss: 0.5759 - val_accuracy: 0.8117

Epoch 00002: saving model to training_2/cp.ckpt
Epoch 3/50
 77/196 [==========>...................] - ETA: 3:06 - loss: 0.4766 - accuracy: 0.8344

In [ ]:
def preprocess_image(x_train,x_test):


  """

    x_train --> training dataframe - type pandas -core-DataFrame

    x_test --> test dataframe - type pandas -core-DataFrame

  
  """

  #preprocess the image scaling
  #Reference : https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
  train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range = 20, horizontal_flip = True, 
                                                                    width_shift_range = 0.1, height_shift_range = 0.1,
                                                                    zoom_range = 0.3, shear_range = 15)
  

  train_generator.fit(x_train)

 
  validation_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

  validation_generator.fit(x_test)

  return train_generator,validation_generator